# Elo Merchant Category Recommendation - LynxKite preprocessor
End date: _2019. february 19._<br/>

This tutorial notebook is the first part of a seriers for [Elo Mechant Category Recommendation](https://www.kaggle.com/c/elo-merchant-category-recommendation) contest organized by Elo, one of the largest payment brands in Brazil. It has built partnerships with merchants in order to offer promotions or discounts to cardholders. The objective of the competition is to identify and serve the most relevant opportunities to individuals, by uncovering signals in customer loyalty. The input files are available from the [download](https://www.kaggle.com/c/elo-merchant-category-recommendation/data) section of the contest:

- **train.csv**,  **test.csv**: list of `card_ids` that can be used for training and testing
- **historical_transactions.csv**: contains up to 3 months' worth of transactions for every card at any of the provided `merchant_ids`
- **new_merchant_transactions.csv**: contains the transactions at new merchants (`merchant_ids` that this particular `card_id` 
has not yet visited) over a period of two months
- **merchants.csv**: contains aggregate information for each `merchant_id` represented in the data set

In [1]:
import gc
import math
import warnings
import datetime
import numpy as np
import pandas as pd
from scipy.stats import iqr
import matplotlib.pyplot as plt

%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Starting memory usage: {:5.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Reduced memory usage: {:5.2f} MB ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def create_date_features(df, source_column, preposition):
    df[preposition + '_year'] = df[source_column].dt.year
    df[preposition + '_month'] = df[source_column].dt.month
    df[preposition + '_day'] = df[source_column].dt.day
    df[preposition + '_hour'] = df[source_column].dt.hour
    df[preposition + '_weekofyear'] = df[source_column].dt.weekofyear
    df[preposition + '_dayofweek'] = df[source_column].dt.dayofweek
    df[preposition + '_quarter'] = df[source_column].dt.quarter
    
    return df

## Train data

In [4]:
df_train = pd.read_csv("input/train.csv")
df_train = reduce_mem_usage(df_train)

print("{:,} records and {} features in train set.".format(df_train.shape[0], df_train.shape[1]))

Starting memory usage:  9.24 MB
Reduced memory usage:  4.04 MB (56.2% reduction)
201,917 records and 6 features in train set.


In [5]:
df_train[:3]

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988


In [ ]:
q1_tr = int(df_train['target'].count()/4)
q3_tr = int(3*df_train['target'].count()/4)

df_train.sort_values(by='target', inplace=True, na_position='last')

print('Range of the target in the training set: {} - {}'.format(df_train['target'].min(), df_train['target'].max()))
print('Interquartile range of target in the training set: {} - {}'.format(df_train['target'].iloc[q1_tr], df_train['target'].iloc[q3_tr]))

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_train['target'].values)
plt.title('Histogram target counts')
plt.xlabel('Count')
plt.xticks(rotation=60)
plt.ylabel('First Active Month')
plt.axvline(df_train['target'].iloc[q1_tr], color="k", linestyle="--")
plt.axvline(df_train['target'].iloc[q3_tr], color="k", linestyle="--")
plt.show()

In [ ]:
df_train.shape[0]

In [ ]:
print('According to Rice\'s rule, the number of bins is {:.0f} (for the whole set)'.format(math.sqrt(df_train.shape[0])*2))
print('According to Rice\'s rule, the number of bins is {:.0f} (without the outliers)'.format(math.sqrt(df_train[df_train['target'] > -30].shape[0])*2))

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_train['target'].values, bins=899)
plt.title('Histogram target counts')
plt.xlabel('Count')
plt.xticks(rotation=60)
plt.ylabel('Target')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_train[df_train['target'] > -30]['target'].values, bins=894)
plt.title('Histogram target counts')
plt.xlabel('Count')
plt.xticks(rotation=60)
plt.ylabel('Target')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_train[df_train['target'] > -30]['target'].values, bins=100)
plt.title('Histogram target counts')
plt.xlabel('Count')
plt.xticks(rotation=60)
plt.ylabel('Target')
plt.show()

Create 100 bins for the target variable.

In [6]:
range = math.fabs(df_train[df_train['target'] > -30]['target'].min()) + df_train[df_train['target'] > -30]['target'].max()
print('The range of the training set without the outliers is {}'.format(range))

The range of the training set without the outliers is 35.578125


In [7]:
print('Minimum value without outliers: {}, width of bin: {}'.format(df_train[df_train['target'] > -30]['target'].min(), range/100))

Minimum value without outliers: -17.609375, width of bin: 0.35578125


In [13]:
def bin_target(value, min_value, bin_width, cutoff_value=-20):
    output = 0
    if value > cutoff_value:
        output = int((value - min_value)/bin_width)

    return output

df_train['target_bin'] = df_train['target'].apply(
    lambda x: bin_target(
        x,
        df_train[df_train['target'] > -30]['target'].min(),
        range/100
    )
)

In [14]:
df_train[:3]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,target_bin
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312,47
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822,50
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988,51


In [15]:
df_train[['card_id', 'target_bin', 'target']][:10]

,card_id,target_bin,target
0,C_ID_92a2005557,47,-0.820312
1,C_ID_3d0044924f,50,0.392822
2,C_ID_d639edf6cd,51,0.687988
3,C_ID_186d6a6901,49,0.142456
4,C_ID_cdbd2c0db2,49,-0.159790
5,C_ID_0894217f2f,51,0.871582
6,C_ID_7e63323c00,50,0.230103
7,C_ID_dfa21fc124,55,2.136719
8,C_ID_fe0fdac8ea,49,-0.065430
9,C_ID_bf62c0b49d,50,0.300049


In [16]:
df_train.to_csv('input/train_binned.csv')

## Transactions

In [ ]:
df_hist_trans = pd.read_csv("input/historical_transactions.csv")
print("{:,} records and {} features in historical transactions set.".format(df_hist_trans.shape[0], df_hist_trans.shape[1]))

df_hist_trans = reduce_mem_usage(df_hist_trans)

In [ ]:
df_new_trans = pd.read_csv("input/new_merchant_transactions.csv")
print("{:,} records and {} features in new transactions set.".format(df_new_trans.shape[0], df_new_trans.shape[1]))

df_new_trans = reduce_mem_usage(df_new_trans)

In [ ]:
df_hist_trans['authorized_flag'] = df_hist_trans['authorized_flag'].map({'N': 0, 'Y': 1})
df_hist_trans['category_1'] = df_hist_trans['category_1'].map({'N': 0, 'Y': 1})
df_hist_trans['category_2'] = pd.to_numeric(df_hist_trans['category_2'])
df_hist_trans['category_3'] = df_hist_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})

In [ ]:
df_new_trans['authorized_flag'] = df_new_trans['authorized_flag'].map({'N': 0, 'Y': 1})
df_new_trans['category_1'] = df_new_trans['category_1'].map({'N': 0, 'Y': 1})
df_new_trans['category_2'] = pd.to_numeric(df_new_trans['category_2'])
df_new_trans['category_3'] = df_new_trans['category_3'].map({'A': 0, 'B': 1, 'C': 2})

In [ ]:
df_hist_trans['purchase_date'] = pd.to_datetime(df_hist_trans['purchase_date'])
df_hist_trans = create_date_features(df_hist_trans, 'purchase_date', 'purchase')

In [ ]:
df_new_trans['purchase_date'] = pd.to_datetime(df_new_trans['purchase_date'])
df_new_trans = create_date_features(df_new_trans, 'purchase_date', 'purchase')

### Slicing

In [ ]:
df_new_trans[:10000].to_csv('input/trans_new_first10k.csv')

In [ ]:
df_hist_trans[:10000].to_csv('input/trans_hist_first10k.csv')

### Aggregation

In [ ]:
df_hist_trans = reduce_mem_usage(df_hist_trans)

In [ ]:
df_new_trans = reduce_mem_usage(df_new_trans)

In [ ]:
def aggregate_transactions(df, prefix):
    agg_funcs = {
        'authorized_flag': ['sum', 'mean'],

        'category_1': ['sum', 'mean'],
        'category_2': ['sum', 'mean'],
        'category_3': ['sum', 'mean'],
        
        'city_id': ['nunique'],
        
        'installments': ['sum', 'median', 'mean', 'max', 'min', 'std'],

        'month_lag': ['min', 'max', 'mean'],

        'state_id': ['nunique'],
        'subsector_id': ['nunique'],

        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_year': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_day': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_hour': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_weekofyear': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_dayofweek': ['mean', 'median', 'max', 'min', 'std'],
        'purchase_quarter': ['mean', 'median', 'max', 'min', 'std']
    }
    df_agg = df.groupby('card_id').agg(agg_funcs)
    df_agg.columns = [prefix + '_'.join(col).strip() for col in df_agg.columns.values]
    df_agg.reset_index(inplace=True)

    df = (df_agg.groupby('card_id').size().reset_index(name='{}transactions_count'.format(prefix)))
    df_agg = pd.merge(df, df_agg, on='card_id', how='left')

    return df_agg

In [ ]:
df_new_trans = aggregate_transactions(df_new_trans, prefix='new_')

In [ ]:
df_new_trans[:3]

In [ ]:
df_new_trans.to_csv('input/trans_new_aggregated.csv')

In [ ]:
df_hist_trans = aggregate_transactions(df_hist_trans, prefix='hist_')

In [ ]:
df_hist_trans[:3]

In [ ]:
df_hist_trans.to_csv('input/trans_hist_aggregated.csv')

## Merchants

In [ ]:
df_merch = pd.read_csv("input/merchants.csv")
print("{:,} records and {} features in merchant set.".format(df_merch.shape[0], df_merch.shape[1]))

df_merch = reduce_mem_usage(df_merch)

In [ ]:
df_merch['category_1'] = df_merch['category_1'].map({'N': 0, 'Y': 1})
df_merch['category_2'] = pd.to_numeric(df_merch['category_2'])
df_merch['category_4'] = df_merch['category_4'].map({'N': 0, 'Y': 1})
df_merch['most_recent_sales_range'] = df_merch['most_recent_sales_range'].map({'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4})
df_merch['most_recent_purchases_range'] = df_merch['most_recent_purchases_range'].map({'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4})

In [ ]:
df_merch.to_csv('input/merchant_parsed.csv')

Garbage

In [ ]:
dropping = ['city_id', 'state_id']
for var in dropping:
    df_merch = df_merch.drop(var, axis=1)

In [ ]:
df_hist_trans = pd.merge(df_hist_trans, df_merch, on='merchant_id', how='left', suffixes=['_trans','_merch'])

In [ ]:
df_new_trans = pd.merge(df_new_trans, df_merch, on='merchant_id', how='left', suffixes=['_trans','_merch'])